In [1]:
import bm25s
from src.preprocessor.preprocessor import TextPreprocessing
from src.preprocessor.post_preprocessor import PostPreprocessing
from src.retrieval.lexical.bm25_retrieval import BM25sRetrieval
from src.preprocessor.utils.dataset_level import read_pickle

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [24]:
text = "Khoản Chi hội thảo trong nước chi các hoạt động mang tính chất nghiên cứu khoa học thuộc Chương trình. Nội dung và mức chi thực hiện theo quy_định tại Điều_2 Điều_3 Điều_4 và Điều_6 Thông tư số 03/2023/TTBTC ngày 10 tháng 01 năm 2023 của Bộ_Tài_chính quy_định lập dự toán quản lý sử dụng và quyết toán kinh phí ngân sách nhà nước thực hiện nhiệm vụ khoa học và công nghệ sau đây viết tắt là Thông tư số 03/2023/TT"
import re
import string
from pyvi import ViTokenizer
from src.preprocessor.vocab.stopwords import STOP_WORDS
from src.preprocessor.vocab.legal_dict import LEGAL_DICT
from src.preprocessor.vocab.duties_dict import DUTIES
from src.preprocessor.vocab.special_terms import SPECIAL_TERMS
from src.preprocessor.vocab.numeral_currency import CURRENCY
from src.preprocessor.utils.sent_level import dupplicated_char_remover, preprocess_pyvi, postprocess_pyvi
from src.preprocessor.legal_processing.legal_terms_tokenize import terms_of_law
from src.preprocessor.legal_processing.duties_tokenize import duties_terms, ner_tokenize

def url_remover(paragraph: str) -> str:
    """Remove URLs within parentheses from text."""
    pattern = r'\([^)]*http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+[^)]*\)'
    
    def replace_url(match):
        content = match.group(0)
        cleaned = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', content)
        return cleaned if cleaned.strip('() ') else ''
        
    return re.sub(pattern, replace_url, paragraph)
def punctuation_remover(paragraph: str) -> str:
    """Remove punctuation marks from text and handle currency conversions."""
    # Loại bỏ dấu ngoặc khỏi số
    paragraph = re.sub(r"\((\d+)\)", r"\1", paragraph)
    paragraph = re.sub(r"\w+\)", " ", paragraph)

    # Tách các từ trong đoạn văn
    words = paragraph.split()
    updated_words = []

    # Xử lý các từ liên quan đến số và tiền tệ
    for item in words:
        # Nếu từ kết thúc bằng ')' hoặc '.' và bắt đầu bằng số từ 1-9, loại bỏ từ đó
        if item.endswith((')', '.')) and item[0] in '123456789':
            continue

        # Thay thế chính xác từ khớp với từ điển CURRENCY
        if item in CURRENCY:
            updated_words.append(CURRENCY[item])
            continue

        # Tìm và thay thế các trường hợp chứa mẫu trong từ điển CURRENCY
        for key, value in CURRENCY.items():
            if key in item:
                item = item.replace(key, f" {value}")
                break

        updated_words.append(item)

    # Kết hợp lại đoạn văn đã được cập nhật
    paragraph = ' '.join(updated_words)

    # Loại bỏ các dấu câu (trừ '/' và '.')
    for punc in string.punctuation:
        if punc == ":":
            paragraph = paragraph.replace(punc, ".")
        elif punc == '-':
            paragraph = paragraph.replace(punc, "")
        elif punc not in ["/", "."]:
            paragraph = paragraph.replace(punc, " ")

    # Loại bỏ khoảng trắng thừa
    return re.sub(r"\s+", " ", paragraph).strip()
def line_breaker_remover(paragraph: str) -> str:
    """Remove line breaks from text."""
    para = re.sub(r"\\n+", ". ", paragraph)
    para = re.sub(r"\n+", ". ", paragraph)
    para = re.sub(r"\.\.\.", " ", para)
    para = re.sub(r'\.{1,}', '.', para)
    return para.replace("  ", " ")
def lowercase_standardizer(paragraph: str) -> str:
    """Convert text to lowercase."""
    return paragraph.lower()
def white_space_remover(paragraph: str) -> str:
        """Remove extra whitespace from text."""
        para = paragraph.replace("  ", " ")
        return re.sub(r"\s{2,}", " ", para).strip()
def legal_text_tokenizer(paragraph: str) -> str:
    """Tokenize legal terms in text."""
    
    for phrase, replacement in LEGAL_DICT.items():
        paragraph = paragraph.replace(phrase, replacement)
    
    paragraph = terms_of_law(paragraph)
    paragraph = duties_terms(paragraph)
    paragraph = dupplicated_char_remover(paragraph)   
    return paragraph
def text_tokenizer(paragraph: str) -> str:
        """Tokenize regular text."""
        paragraph = ner_tokenize(paragraph)
        for phrase, replacement in SPECIAL_TERMS.items():
            paragraph = paragraph.replace(phrase, replacement)
        paragraph = preprocess_pyvi(paragraph)
        paragraph = ViTokenizer.tokenize(paragraph)
        paragraph = postprocess_pyvi(paragraph)
        return paragraph

text_tokenizer(legal_text_tokenizer(white_space_remover(lowercase_standardizer(line_breaker_remover(punctuation_remover(url_remover(text)))))))

khoản chi hội thảo trong nước chi các hoạt động mang tính chất nghiên cứu khoa học thuộc chương trình. nội dung và mức chi thực hiện theo quy_định tại Điều_2 Điều_3 Điều_4 và Điều_6 thông tư số {03/2023/ttbtc} ngày 10 tháng 01 năm 2023 của Bộ_Tài_chính quy_định lập dự toán quản lý sử dụng và quyết toán kinh phí ngân sách nhà nước thực hiện nhiệm vụ khoa học và công nghệ sau đây viết tắt là thông tư số {03/2023/tt}
-------


'khoản chi_hội_thảo trong nước chi các hoạt_động mang tính_chất nghiên_cứu khoa_học thuộc chương_trình . nội_dung và mức chi thực_hiện theo quy_định tại Điều_2 Điều_3_Điều_4 và Điều_6 thông_tư số 03/2023/ttbtc ngày 10 tháng 01 năm 2023 của Bộ_Tài_chính quy_định lập dự_toán quản_lý sử_dụng và quyết_toán kinh_phí ngân_sách nhà_nước thực_hiện nhiệm_vụ khoa_học và công_nghệ sau đây viết tắt là thông_tư số 03/2023/tt'

In [79]:
test = 'khoản chi_hội_thảo trong nước chi các hoạt_động mang tính_chất nghiên_cứu khoa_học thuộc chương_trình . nội_dung và mức chi thực_hiện theo quy_định tại Điều_2 Điều_3_Điều_4 và Điều_6 thông_tư số 03/2023/ttbtc ngày 10 tháng 01 năm 2023 của Bộ_Tài_chính quy_định lập dự_toán quản_lý sử_dụng và quyết_toán kinh_phí ngân_sách nhà_nước thực_hiện nhiệm_vụ khoa_học và công_nghệ sau đây viết tắt là thông_tư số 03/2023/tt'
def handle_rules(text: str):
        """Handle rules based text processing with specific formatting.
        
        Args:
            text: Input text containing rules
            
        Returns:
            Tuple of processed text parts or original text if no rules apply
        """
        numbers = [str(i) for i in range(1, 1000)]
        rules = ['Khoản', 'Điều', 'Điểm', 'Chương', 'Cấp', 'Hạng', 'Mục']
        words = text.split("_")
        
        if len(words) == 4 and any(rule in text for rule in rules):
            return f"{words[0]}_{words[1]}", f"{words[-2]}_{words[-1]}"
        
        elif len(words) == 6 and any(rule in text for rule in rules):
            return f"{words[0]}_{words[1]}", f"{words[2]}_{words[3]}", f"{words[-2]}_{words[-1]}"
        
        elif len(words) > 2 and words[-2] in rules and words[-1] in numbers:
            return (f"{words[-2]}_{words[-1]}",
                f"{words[:-2]}",)
        else:
            return text
text_test = []
for i in test.split(" "):
    i = handle_rules(i)
    if isinstance(i, str):
        text_test.append(i)
    elif isinstance(i, tuple):
        text_test.extend([j for j in i])
final_text = []
for i in text_test:
    if i not in STOP_WORDS and i not in LEGAL_DICT and i not in DUTIES and i not in SPECIAL_TERMS:
        if len(i.split("_")) == 3:
            before, after = i.split("_")[0] ,"_".join(i.split("_")[1:])
            final_text.append(before)
            final_text.append(after)
        else:
            final_text.append(i)
    else:
        final_text.append(i)

" ".join(final_text)

'khoản chi hội_thảo trong nước chi các hoạt_động mang tính_chất nghiên_cứu khoa_học thuộc chương_trình . nội_dung và mức chi thực_hiện theo quy_định tại Điều_2 Điều_3 Điều_4 và Điều_6 thông_tư số 03/2023/ttbtc ngày 10 tháng 01 năm 2023 của Bộ_Tài_chính quy_định lập dự_toán quản_lý sử_dụng và quyết_toán kinh_phí ngân_sách nhà_nước thực_hiện nhiệm_vụ khoa_học và công_nghệ sau đây viết tắt là thông_tư số 03/2023/tt'

In [61]:
from src.preprocessor.vocab.stopwords import STOP_WORDS
from src.preprocessor.vocab.legal_dict import LEGAL_DICT
from src.preprocessor.vocab.duties_dict import DUTIES
from src.preprocessor.vocab.special_terms import SPECIAL_TERMS
from src.preprocessor.vocab.numeral_currency import CURRENCY

STOP_WORDS = list(STOP_WORDS)
LEGAL_DICT = list(LEGAL_DICT.values())
DUTIES = list(DUTIES.values())
SPECIAL_TERMS = list(SPECIAL_TERMS.values())
for i in tesss:
    if i not in STOP_WORDS and i not in LEGAL_DICT and i not in DUTIES and i not in SPECIAL_TERMS:
        if len(i.split("_")) == 3:
            before, after = i.split("_")[0] ,"_".join(i.split("_")[1:])
            

chi hội_thảo


In [2]:
queries = read_pickle("./data/processed/queries.pkl")
query = "Tiêu chuẩn về đạo đức nghề nghiệp giáo viên mầm non theo Thông tư 08/2023/TT-BGDĐT ra sao?"
corpus = read_pickle("./data/processed/corpus.pkl")
corpusv2 = [j for i, j in corpus.items()]
bm25 = BM25sRetrieval(query)

In [3]:
a, b = bm25.retrieval_module(corpusv2)

Split strings:   0%|          | 0/261772 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/261772 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/261772 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Đây là doc_tokens: Khoản đầu_tư mua trái_phiếu quy_định Khoản_6 Điều_1 Nghị_định 65/2022/NĐCP Khoản_1 Điều_3 Nghị_định 08/2023/NĐCP 05 3 2023 Chính_phủ sửa_đổi bổ_sung ngưng hiệu_lực thi_hành Nghị_định quy_định chào_bán giao_dịch trái_phiếu doanh_nghiệp riêng_lẻ thị_trường chào_bán trái_phiếu doanh_nghiệp thị_trường quốc_tế gọi tắt Nghị_định 08/2023/NĐ .
Đây là doc_tokens: Khoản quyết_liệt đồng_bộ hiệu_quả nhiệm_vụ giải_pháp giao nghị_quyết phiên họp Chính_phủ thường_kỳ Chỉ_thị 08/CTTTg 23 3 2023 Công_điện 123/CĐTTg 10 3 2023 Thủ_tướng_Chính_phủ văn_bản chỉ_đạo Lãnh_đạo Chính_phủ đôn_đốc phân_bổ giải_ngân vốn_đầu_tư công 2023 coi giải_ngân kế_hoạch vốn_đầu_tư 2023 nhiệm_vụ chính_trị trách_nhiệm toàn_diện Chính_phủ Thủ_tướng_Chính_phủ chậm giải_ngân phấn_đấu tỷ_lệ giải_ngân 2023 95 kế_hoạch giao .
Đây là doc_tokens: Khoản Chi hội_thảo chi hoạt_động tính_chất nghiên_cứu khoa_học Chương_trình . Nội_dung chi quy_định Điều_2 Điều_3 Điều_4 Điều_6 Thông_tư 03/2023/TTBTC 10 01 2023 Bộ_Tài_chín